In [1]:
import os
import numpy as np
import pandas as pd
from typing import Union
from sklearn.manifold import TSNE
import seaborn as sns

import json
import numpy as np
import pandas as pd
from typing import Union
import matplotlib.pyplot as plt
import flowkit as fk

In [2]:
def compute_dist(input_a,input_b,input_c): # diff
    get_dist_ab=float(np.sqrt(sum(np.power(input_a-input_b,2))))
    get_dist_ac=float(np.sqrt(sum(np.power(input_a-input_c,2))))
    return abs(get_dist_ab-get_dist_ac)

def compute_single_dist(input_a,input_c): # same
    get_dist_ac=float(np.sqrt(sum(np.power(input_a-input_c,2))))
    return get_dist_ac

def compute_age(input_a,input_b,input_c): # diff
    return abs(input_b-abs(input_a-input_c))

def compute_single_age(input_a,input_c):# same
    return abs(input_a-input_c)


def random_feats(X: np.ndarray,
                gamma: Union[int, float] = 1,
                frequency_seed: int = None,sub_dim:int=None):
    scale = 1 / gamma

    if (frequency_seed is not None):
        np.random.seed(frequency_seed)
        W = np.random.normal(scale = scale, size = (X.shape[1], sub_dim))
    else:
        W = np.random.normal(scale = scale, size = (X.shape[1], sub_dim))

    XW = np.dot(X, W)
    sin_XW = np.sin(XW)
    cos_XW = np.cos(XW)
    phi = np.concatenate((cos_XW, sin_XW), axis=1)

    return phi

def preread_file(in_rff_dict,in_path,in_fcs_path,inget_marker,fcs_csv,in_median_max_pooling):
    get_file_name=pd.read_csv(in_path)
    for data_line in get_file_name['file_id']:
        if fcs_csv == 'fcs':
            sample = fk.Sample(in_fcs_path+str(data_line),ignore_offset_error=True)
            dm = sample._get_raw_events()
            CNames_pns = np.array(sample.pns_labels)
            data_dff=pd.DataFrame(dm,columns=CNames_pns)
            df_single=np.arcsinh(1./5 * data_dff[inget_marker])
        else:
            df_single = pd.read_csv(in_fcs_path+str(data_line))
            dfv_0_arr=np.array(df_single[inget_marker])
            df_single=np.arcsinh(1./5 * dfv_0_arr)
        gettrf=random_feats(X=df_single,sub_dim=400) 
        if in_median_max_pooling=='max':
            gettrf=np.max(gettrf, axis=0)
        elif in_median_max_pooling=='mean':
            gettrf=np.mean(gettrf, axis=0)
        else:
            gettrf=np.median(gettrf, axis=0)
        
        in_rff_dict[data_line]=gettrf
    return in_rff_dict


def read_files(input_path_base,input_path,input_id_lst,input_label_lst,input_co_lst,input_rff,inrff_vector, in_covariate_label):
    with open(input_path_base+input_path, 'r') as j:
        data = j.read()
    data=data.split('\n')
    # print(data)
    
    for data_line in data:
        data_split=data_line.split(',')
        if data_split[0]=='fcs_filename':
            covariate_label_index=data_split.index(in_covariate_label)
            continue
        input_id_lst.append(data_split[0])
        input_label_lst.append(data_split[1])
        input_co_lst.append(data_split[covariate_label_index]) 
        input_rff.append(inrff_vector[data_split[0]])
    return input_id_lst,input_label_lst,input_co_lst,input_rff

#-----------------------------------------------------

def generate_cytocoset_RFF(preread_path, sub_directory, triplet_file_name, all_end_name, covariate_label, median_max_pooling):

    continuous_covariate = False
    #decide the cell data is csv or fcs
    if os.path.exists(preread_path+'/fcs_info.csv'):
        rawdata_format='fcs'
    elif os.path.exists(preread_path+'/csv_info.csv'):
        rawdata_format='csv'
    else:
        rawdata_format=None
    # create file directory
    for pick_end in all_end_name:
        newpath = preread_path+'/' + triplet_file_name + '_' + pick_end
        if not os.path.exists(newpath):
            os.makedirs(newpath)


    path_base=preread_path+"/" + sub_directory + "/all/"
    get_marker=pd.read_csv(path_base + 'marker.csv')
    get_marker=list(get_marker.columns)

    rff_dict={}
    # generate RFF for each sample at once
    rff_dict=preread_file(rff_dict,preread_path+'/' + rawdata_format + '_info.csv',preread_path+'/' + sub_directory + '/all/',get_marker,rawdata_format,median_max_pooling)

    age_info=pd.read_csv(preread_path+'/' + rawdata_format + '_info.csv')

    # decide balance sample number of train and test set
    uni_id={}
    for i,j in zip(age_info['file_id'],age_info['condition']):
        uni_id[i.split('.')[0]]=j

    ct_id=[]
    co_id=[]

    for i in uni_id.keys():
        if uni_id[i]==0:
            ct_id.append(i)
        elif uni_id[i]==1:
            co_id.append(i)

    get_number=min(int(len(co_id)/2),int(len(ct_id)/2))
    print("sample number:",get_number)

    #run for each trials for both train set RFF data and test set RFF data
    for balence_sample_num in range(0, get_number*4, get_number*2):
        for file_name in all_end_name:

            id_lst=[]
            label_lst=[]
            co_lst=[]
            rff=[]

            # get revelent information in train/test set
            id_lst,label_lst,co_lst,rff=read_files(path_base,"train_labels_"+file_name+".csv",id_lst,label_lst,co_lst,rff,rff_dict,covariate_label)
            id_lst,label_lst,co_lst,rff=read_files(path_base,"test_labels_"+file_name+".csv",id_lst,label_lst,co_lst,rff,rff_dict,covariate_label)

            # decide if is concinuous covariate
            if len(np.unique(co_lst))>2:
                continuous_covariate=True

            if continuous_covariate:
                med_co=np.median(age_info[covariate_label])


            json_file_path = preread_path + "/filenames_"+file_name+".json"

            # dealing with sample order in sheet 
            with open(json_file_path, 'r') as j:
                data = j.read()
            data=data.split('\n')
            json_name=[]
            match_map=[]
            for name_lst in data:
                json_name.append(name_lst.split('/')[1])
                match_map.append(id_lst.index(name_lst.split('/')[1]))

            triplet=[]
            id_lst=list(np.array(id_lst)[np.array(match_map)])
            label_lst=list(np.array(label_lst)[np.array(match_map)])
            co_lst=list(np.array(co_lst)[np.array(match_map)])
            rff=list(np.array(rff)[np.array(match_map)])


            start = balence_sample_num
            end = balence_sample_num + get_number*2

            if start == 0 and end == get_number * 2:
                train_test='trainval'
            else:
                train_test='test'

            #generate RFF in discrete covariate--------
            if not continuous_covariate:
                print('discrete covariate triplet '+ train_test + 'set in ' + file_name + ' trials')
                for d1 in range(start,end,1):
                    for d2 in range(start,end,1):
                        for d3 in range(start,end,1):
                            num_decide=[]
                            num_decide.extend((d1,d2,d3))
                            if len(np.unique(num_decide))==3:
                                if (co_lst[d1]==co_lst[d2] and co_lst[d2]==co_lst[d3]) is False:
                                    if (co_lst[d1]==co_lst[d2]):
                                        if d1<d2:
                                            triplet.append(str(d1)+' '+str(d3)+' '+str(d2)) # corr
                                        else:
                                            triplet.append(str(d2)+' '+str(d3)+' '+str(d1)) # corr
                                    elif(co_lst[d2]==co_lst[d3]):
                                        if d2<d3:
                                            triplet.append(str(d2)+' '+str(d1)+' '+str(d3)) # corr
                                        else:
                                            triplet.append(str(d3)+' '+str(d1)+' '+str(d2)) # corr
                                    elif(co_lst[d1]==co_lst[d3]):
                                        if d1<d3:
                                            triplet.append(str(d1)+' '+str(d2)+' '+str(d3)) # corr
                                        else:
                                            triplet.append(str(d3)+' '+str(d2)+' '+str(d1)) # corr
            else:
                # generate RFF in continuous covariate--------
                print('continue covariate triplet '+ train_test + 'set in ' + file_name + ' trials')
                for d1 in range(start,end,1):
                    for d2 in range(start,end,1):
                        for d3 in range(start,end,1):
                            num_decide=[]
                            num_decide.extend((d1,d2,d3))
                            # print(co_lst[d1],co_lst[d2],co_lst[d3])
                            check_bool=[int(co_lst[d1])<med_co,int(co_lst[d2])<med_co,int(co_lst[d3])<med_co]
                            # print(check_bool)
                            if len(np.unique(num_decide))==3:
                                if check_bool[0]==check_bool[2] and check_bool[0]!=check_bool[1]:
                                    triplet.append(str(d1)+' '+str(d2)+' '+str(d3)) # corr

            triplet=np.unique(triplet)

            final_single_trip=[] #same
            total_dist=[] # diff

            if continuous_covariate:
                final_age_diff=[]
                total_age_diff=[]
            # compute each triplet difference 
            for i in triplet:
                split_i=i.split(" ")
                trip_dist=compute_dist(np.array(rff[int(split_i[0])]),np.array(rff[int(split_i[1])]),np.array(rff[int(split_i[2])])) #diff
                trip_single_dist=compute_single_dist(np.array(rff[int(split_i[0])]),np.array(rff[int(split_i[2])]))# same
                total_dist.append(trip_dist)
                final_single_trip.append(trip_single_dist)
                if continuous_covariate:
                    total_age_diff.append(compute_age(int(co_lst[int(split_i[0])]), int(co_lst[int(split_i[1])]), int(co_lst[int(split_i[2])])))
                    final_age_diff.append(compute_single_age(int(co_lst[int(split_i[0])]),int(co_lst[int(split_i[2])])))
                

            if continuous_covariate:
                final_single_trip=list(np.array(final_single_trip)/(np.array(final_age_diff)+1))
                total_dist=list(np.array(total_dist)/(np.array(total_age_diff)+1))
            # based on difference of each triplet, save in different threhold list file
            for same_pick in range(2,11,2):
                for diff_pick in range(2,11,2):
                    same_percentage=same_pick/10
                    diff_percentage=diff_pick/10

                    final_trip=[]
                    qua_same=np.quantile(final_single_trip, same_percentage)
                    qua_diff=np.quantile(total_dist, 1 - diff_percentage)
                    for i in range(0,len(triplet),1):
                        if total_dist[i]>=qua_diff and final_single_trip[i]<=qua_same:
                            final_trip.append(triplet[i])

                    with open(preread_path + '/' + triplet_file_name + '_'+file_name+'/' + covariate_label + '_tripletlist_subpick_'+train_test+'_rffmax_same'+str(same_percentage)+'_diff'+str(diff_percentage)+'.txt', 'w') as f:
                        for line in final_trip:
                            f.write(line)
                            f.write('\n')

                    # print(same_percentage,diff_percentage,len(final_trip))

In [3]:
dataset_path='/Users/cookie/Documents/UNC/research/lung_cancer/data_lung_1' #path of dataset
dataset_main_directory='lung_fcs' # main file name of dataset
create_triplet_file='tripletlists_check' # triplet name
trials_list=['4','44'] # seed list
covariate_title='sex' # covariate name
pooling='median' #median or max

generate_cytocoset_RFF(dataset_path, dataset_main_directory, create_triplet_file, trials_list, covariate_title, pooling)

/Users/cookie/miniconda3/envs/py37/lib/python3.7/site-packages/flowio/flowdata.py:344: UserWarning: FCS file 1_1_02_b_SD_CD45_.fcs reported incorrect data offset. Attempting to parse data section, but event data should be reviewed before trusting this file.
  warn(warn_msg)
/Users/cookie/miniconda3/envs/py37/lib/python3.7/site-packages/flowio/flowdata.py:344: UserWarning: FCS file 1_2_03_b_PD_CD45_.fcs reported incorrect data offset. Attempting to parse data section, but event data should be reviewed before trusting this file.
  warn(warn_msg)
/Users/cookie/miniconda3/envs/py37/lib/python3.7/site-packages/flowio/flowdata.py:344: UserWarning: FCS file 1_3_04_b_PD_CD45_.fcs reported incorrect data offset. Attempting to parse data section, but event data should be reviewed before trusting this file.
  warn(warn_msg)
/Users/cookie/miniconda3/envs/py37/lib/python3.7/site-packages/flowio/flowdata.py:344: UserWarning: FCS file 1_4_05_b_PD_CD45_.fcs reported incorrect data offset. Attempting t

sample number: 6
discrete covariate triplet trainvalset in 4 trials
discrete covariate triplet trainvalset in 44 trials
discrete covariate triplet testset in 4 trials
discrete covariate triplet testset in 44 trials
